<a href="https://colab.research.google.com/github/EdithPraiselin/-RECOMMENDATION-SYSTEM/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505180 sha256=1a5085362971192ac8665ba71232957ef5b081b341cea88a54f5a74904f869fa
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse, mae

# Update Dataset Paths
ratings_path = "/ratings.csv"
movies_path = "/movies.csv"

# Load the MovieLens 20M Dataset (CSV Format)
ratings = pd.read_csv(ratings_path, usecols=["userId", "movieId", "rating"])
movies = pd.read_csv(movies_path, usecols=["movieId", "title"])
movies_dict = dict(zip(movies["movieId"], movies["title"]))

# Prepare Data for Surprise
reader = Reader(line_format="user item rating", sep=",", rating_scale=(0.5, 5.0))
dataset = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], reader)

# Count ratings per movie
movie_counts = ratings["movieId"].value_counts()

# Keep only movies with at least 10 ratings
popular_movies = movie_counts[movie_counts >= 10].index
filtered_ratings = ratings[ratings["movieId"].isin(popular_movies)]

# Split Data into Training and Testing Sets
trainset, testset = train_test_split(dataset, test_size=0.1)

# Train the SVD Model
algo = SVD(n_factors=150, lr_all=0.007, reg_all=0.02)
algo.fit(trainset)

# Evaluate Model Performance
predictions = algo.test(testset)
print("\nEvaluation Metrics:")
print("RMSE:", rmse(predictions))
print("MAE:", mae(predictions))

# Generate Top-N Recommendations for Each User
def get_top_n_recommendations(predictions, n=5):
    top_n = {}
    for uid, iid, _, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]  # Keep Top-N movies

    return top_n

top_recommendations = get_top_n_recommendations(predictions)

# Convert Movie IDs to Titles and Display Sample Recommendations
def display_recommendations(user_id, top_recommendations):
    if user_id not in top_recommendations:
        print(f"No recommendations found for User {user_id}")
        return

    print(f"\n🎬 Top 5 Recommendations for User {user_id}:")
    for movie_id, score in top_recommendations[user_id]:
        print(f"{movies_dict.get(movie_id, 'Unknown Movie')} (Predicted Rating: {score:.2f})")

# Show recommendations for 5 random users
import random
random_users = random.sample(list(top_recommendations.keys()), 5)
for user in random_users:
    display_recommendations(user, top_recommendations)



Evaluation Metrics:
RMSE: 0.7799
RMSE: 0.779920347981249
MAE:  0.5923
MAE: 0.5922932791166956

🎬 Top 5 Recommendations for User 125520:
Independence Day (a.k.a. ID4) (1996) (Predicted Rating: 4.43)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) (Predicted Rating: 4.39)
Psycho (1960) (Predicted Rating: 4.37)
Terminator, The (1984) (Predicted Rating: 4.34)
Singin' in the Rain (1952) (Predicted Rating: 4.18)

🎬 Top 5 Recommendations for User 49647:
Kronos (a.k.a. Captain Kronos: Vampire Hunter) (1973) (Predicted Rating: 3.44)
Children of the Damned (1963) (Predicted Rating: 3.12)
Abyss, The (1989) (Predicted Rating: 2.96)
Trippin' (1999) (Predicted Rating: 2.42)

🎬 Top 5 Recommendations for User 57346:
One Flew Over the Cuckoo's Nest (1975) (Predicted Rating: 5.00)
Pulp Fiction (1994) (Predicted Rating: 5.00)
Crying Game, The (1992) (Predicted Rating: 4.84)
Hoop Dreams (1994) (Predicted Rating: 4.57)
Beyond Silence (Jenseits der Stille) (1996) (Predicted R